# 帮手程序说明

这一节，我们暂停推导公式的步伐，先了解一下自定义的帮手程序，以及 Hacking 帮手程序的一些技巧和一些程序问题．尽管这些程序在现在或将来会有程序文档，但这可以看作是一份非正式的、以及有少许范例的程序文档．

尽管与后面的公式推导无关，但由于以后经常调用帮手程序，因此这一节仍然重要．

帮手程序的路径应当在 `PYTHONPATH` 中．

## Jupyter 笔记本的库载入

默认下，Jupyter 笔记本的库不重复载入．这意味着在导入 (import) 库之后，如果你更改了库文件的代码，或者通过一些手段更改库文件的配置，你必须重启当前 Jupyter 笔记本的内核 (kernel) 以更新这些库的调用．

但如果执行下述代码，你可以通过再次导入库来更新库的调用，避免重启内核．这在用 Jupyter 调试代码的过程中会非常方便．

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import os, sys
from pyscf import scf, gto, lib, grad, hessian, dft
from functools import partial

np.einsum = partial(np.einsum, optimize=["greedy", 1024 ** 3 * 2 / 8])
np.set_printoptions(5, linewidth=150, suppress=True)

In [ ]:
mol = gto.Mole()
mol.atom = """
O  0.0  0.0  0.0
O  0.0  0.0  1.5
H  1.5  0.0  0.0
H  0.0  0.7  1.5
"""
mol.basis = "6-31G"
mol.verbose = 0
mol.build()

grids = dft.gen_grid.Grids(mol)
grids.atom_grid = (99, 590)
grids.becke_scheme = dft.gen_grid.stratmann
grids.prune = None
grids.build()

nmo = nao = mol.nao
natm = mol.natm
nocc = mol.nelec[0]
nvir = nmo - nocc
so = slice(0, nocc)
sv = slice(nocc, nmo)
sa = slice(0, nmo)

## 帮手程序 utilities

In [ ]:
from utilities import val_from_fchk, timing, timing_level

### `val_from_fchk`

这个小程序可以从由 formchk 生成的文件中，通过传入数值的名称 (譬如 `Cartesian Gradient`)，返回其对应的数值．这个数值可以是一个值 (譬如能量)，也可以是一个向量 (压平后的梯度、Hessian 等)．我们在以前核验 Gaussian 与 PySCF 的 RHF 与 B3LYP 能量时已经使用过该函数．

### `timing_level` 与 `timing`

这个小程序被用于对函数作挂墙时间 (wall time) 测评；挂墙时间与实际 CPU 运行时间 (CPU time) 不同．这个函数可以用作修饰符，这在 [Python 技巧](python_intro.ipynb#修饰符) 中已经有所介绍．

该函数可以传入一个数值参数，该数值参数是输出级别．若参数的值越低，那么程序越有可能输出挂墙时间．

In [ ]:
@timing_level(0)
def large_loop_1():
    np.array([2 * i for i in range(10000)]).sum()
large_loop_1()

In [ ]:
@timing_level(2)
def large_loop_2():
    np.array([2 * i for i in range(10000)]).sum()
large_loop_2()

`timing` 等价于 `timing_level(2)`：

In [ ]:
@timing
def large_loop_3():
    np.array([2 * i for i in range(10000)]).sum()
large_loop_3()

我们可以通过更改 Bash 环境变量 `LOGLEVEL` 来更改默认输出级别；但我们需要彻底重新导入库文件才能让更新的输出级别生效，并且已定义的函数仍然按照原先的输出级别进行输出：

In [ ]:
os.environ["LOGLEVEL"] = "2"
del sys.modules["utilities"]
from utilities import timing, timing_level

In [ ]:
large_loop_2()

In [ ]:
@timing_level(2)
def large_loop_4():
    np.array([2 * i for i in range(10000)]).sum()
large_loop_4()

In [ ]:
@timing
def large_loop_5():
    np.array([2 * i for i in range(10000)]).sum()
large_loop_5()

在下文中，我们希望避免太多输出．

In [ ]:
os.environ["LOGLEVEL"] = "0"
del sys.modules["utilities"]
from utilities import timing, timing_level

## RHF 梯度帮手 `HFHelper`

`HFHelper` 类有众多与 RHF 梯度有关的量．

In [ ]:
from hf_helper import HFHelper

关于这个类内部属性 (property) 的意义，许多将在以后的文档中，或者程序文档中再作详细说明．我们在这里只对其基本的使用方式与结构作简单介绍．

### 初始化与基本信息

我们可以通过代入 `gto.Mole` 类来初始化该帮手类：

In [ ]:
hfh = HFHelper(mol)

初始化过程会自动地进行自洽场计算，并且保存分子轨道系数、密度、占据数、轨道能等基本信息．这些信息一旦被保存，就不能被修改：

In [ ]:
hfh.C.shape

In [ ]:
hfh.C = np.random.random(hfh.C.shape)

如果希望进行非自洽计算，从其它类中获得分子轨道、密度、占据数、轨道能等信息，可以通过传入 `init_scf=False` 来避免类初始化时进行的自洽场计算：

In [ ]:
nch = HFHelper(mol, init_scf=False)
nch.C = hfh.C
nch.D = hfh.D
nch.e = hfh.e
nch.mo_occ = hfh.mo_occ

在 `nch` 中一旦这些变量被设定，一样不能再被更改：

In [ ]:
nch.C = hfh.C

### 属性的调用与内部过程

HFHelper 有许多属性，除了上述四种属性可以被写入一次，其余的属性都不可被写入．以轨道数为例：

In [ ]:
hfh.nmo

In [ ]:
hfh.nmo = 22

许多属性的计算需要时间；在 HFHelper 中，这些需要被计算的属性在第一次被调用时，会储存在类内弱保护变量中；以后再被调用时，则直接访问弱保护变量而不再计算．

这里的“弱保护”是指变量名首字符为单下划线的变量，PEP8 下，这些变量不应被类外程序所访问．相对地，“强保护”是指变量名前两字符为双下划线的变量，Python 解释器不允许类外程序方便地直接访问这些变量．

以 MO 基组 Fock 矩阵为例．其属性名为 `F_0_mo`，而弱保护变量为 `_F_0_mo`；对应的 AO 基组 Fock 矩阵的弱保护变量为 `_F_0_ao`：

In [ ]:
print(type(hfh._F_0_ao))
print(type(hfh._F_0_mo))

当第一次调用 `F_0_mo` 时，程序会计算 MO 基组下的 Fock 矩阵；而计算 MO 基组下的 Fock 矩阵也需要 AO 基组的 Fock 矩阵，以此 AO 基组的 Fock 矩阵也一并被计算．这一步尽管耗时很低，但还是费一些 CPU 时间：

In [ ]:
%%time
F_0_mo = hfh.F_0_mo

同时，弱保护变量就会被保存为 NumPy 数组：

In [ ]:
print(type(hfh._F_0_ao))
print(type(hfh._F_0_mo))

第二次调用时，耗时将明显变低，因为第二次调用仅仅是返回类内变量 `_F_0_mo`：

In [ ]:
%%time
F_0_mo = hfh.F_0_mo

这么构造程序中的变量后，我们就不必担心是否在 HFHelper 类初始化时，是否多算或少算了一些属性，避免效率与程序正确性的核验；同时，我们避免了大多数属性的重新计算，并且几乎不影响代码的阅读与书写的体验．

## GGA 梯度帮手 `GGAHelper`

`GGAHelper` 类有众多与 Restricted GGA 梯度有关的量．

In [ ]:
from gga_helper import GGAHelper

该类继承自 `HFHelper` 类，因此，绝大多数的属性都被毫无更改地复制了过来；即使有一些属性与 `HFHelper` 不同，但属性作为量化物理量的意义、使用与传参等等也还是一致．譬如我们要获得系数矩阵，这与 `HFHelper` 几乎完全相同；不同之处是在初始化 `GGAHelper` 类时，需要额外引入泛函字符串、以及格点信息．

In [ ]:
ggah = GGAHelper(mol, "b3lypg", grids)
ggah.C.shape

## 格点助手 `GridHelper`

`GridHelper` 会生成与泛函核无关的格点信息，包括轨道、密度、密度原子坐标梯度、权重等信息．它将生成完整的格点，因此对内存的消耗非常严重；在对大体系进行计算时，这不是可取的方案．但对于小体系而言，格点助手可以方便调试．同时，格点助手不使用属性；在类的初始化过程中，所有的格点都会生成．

In [ ]:
from grid_helper import GridHelper

In [ ]:
grdh = GridHelper(mol, grids, ggah.D)

In [ ]:
grdh.ao_2.shape

## 泛函核助手 `KernelHelper`

`KernelHelper` 会依据当前的泛函名称生成泛函核的格点信息．

In [ ]:
from grid_helper import KernelHelper

In [ ]:
kerh = KernelHelper(grdh, "b3lypg")

下面就是双氧水分子的交换相关能：

In [ ]:
(kerh.exc * grdh.rho_0).sum()

## 格点迭代器 `GridIterator`

`GridIterator` 与 `GridHelper` 一样，可以处理格点积分；但当格点非常大时，它会将格点分成小份，以允许我们在较低的内存下处理每份的贡献，并最终加和．在大体系下，这是更可取的方案，但也意味着每次计算都需要重新生成格点．这种方案以牺牲计算效率换取内存空间．

同时，我们指出 `GridIterator` 采用类似于 `HFHelper` 但不同于 `GridHelper` 和 `KernelHelper` 的方法，即属性的方法，来生成格点．这意味着，除了在每次迭代过程时会生成原子轨道格点之外，剩余的格点都是只在需要的时刻计算．

In [ ]:
from grid_iterator import GridIterator

实例化的 `GridIterator` 可以当作迭代器调用．下面的程序可以处理双氧水的交换相关能的计算：

In [ ]:
grdit = GridIterator(mol, grids, ggah.D)
exc = 0
for grdh in grdit:
    kerh = KernelHelper(grdh, "b3lypg")
    exc += (kerh.exc * grdh.rho_0).sum()
    print(exc, grdh.rho_0.shape)

我们会发现，上述代码的调用过程，除了使用了迭代器之外，其余的代码其实与使用 `GridHelper` 类的工作方式是一致的．因此，使用 `GridHelper` 类来编写的代码应当可以很快地移植到 `GridIterator` 类的程序上．

`GridIterator` 与 `KernelHelper` 类都需要预先定义导数的阶数．在处理 GGA 的二阶梯度性质时，需要额外传入参数 `deriv=3`．

## 数值梯度助手 `NumericDiff`

`NumericDiff` 可以分别依靠能量、梯度形式的值或张量，生成梯度、Hessian 形式的对应的数值导数或导数张量．

In [ ]:
from numeric_helper import NumericDiff

实例化一个数值梯度助手需要分子构型、获得能量或梯度值的函数，以及一些参数．譬如，我们可以通过下述代码生成 RHF 能量的梯度：

In [ ]:
rhf_grad = NumericDiff(mol, lambda mol: HFHelper(mol).eng).get_numdif()
np.allclose(rhf_grad, hfh.scf_grad.kernel())

我们还可以通过 RHF 能量梯度的导数获得 RHF 的 Hessian；但由于这是 Hessian 型的导数，因此需要加上选项 `deriv=2`：

In [ ]:
rhf_hess = NumericDiff(mol, lambda mol: HFHelper(mol).scf_grad.kernel(), deriv=2).get_numdif()
np.allclose(rhf_hess, hfh.scf_hess.kernel())

## 非自洽计算助手 `NCGGAEngine`

这份文档的最主要目的是得到非自洽泛函的梯度与 Hessian．这通过 `NCGGAEngine` 实现．

In [ ]:
from ncgga_engine import NCGGAEngine

对于 `NCGGAEngine`，我们需要输入的参数是一个自洽场的助手，与一个未经过 SCF 计算的非自洽泛函的助手．其中，自洽泛函助手可以是 `HFHelper` 或 `GGAHelper`，而非自洽泛函助手只可以是 `GGAHelper`．下述的非自洽助手实例是 HF-B3LYP 非自洽体系．

In [ ]:
nch = GGAHelper(mol, "b3lypg", grids, init_scf=False)
nceng = NCGGAEngine(hfh, nch)

我们可以依次获得上述 HF-B3LYP 泛函的能量、一阶梯度与二阶梯度．

In [ ]:
nceng.get_E_0()

In [ ]:
nceng.get_E_1()

In [ ]:
nceng.get_E_2()